# Face Attendance System Test-file

##  Generate Known faces & labels

In [9]:
## generate.py

"""NOTE:
    - Collect images with labels in it
    - store them to Input_images folder
    - Run generate.py to create known faces-encoding & label for further process
"""

import cv2
import numpy as np
import os
import face_recognition as fr


def generate(path):
    """ Takes images-path(with names) & makes known-face database(extracts labels from image , extract faces encodings) & Also stores labels & face-encodings to '.npy' file """
    labels, encodings = [], []

    if len(os.listdir(path)) != 0:
        # print("Found {} images ")
        for image in os.listdir(path):
            if image.endswith(".jpg") or image.endswith(".png"):
                img = cv2.imread(os.path.join(path, image))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # BGR to RGB channel format
                encode = fr.face_encodings(img)[0]  # extract face encodings from image
                encodings.append(encode)
                labels.append(os.path.splitext(image)[0])  # list of known-labels

        ## convert & store, names & face encodings to ".npy"
        np.save(r"D:\Projects\face attendance system\Backend\known_faces_database\known_face_encodings.npy", encodings)
        np.save(r"D:\Projects\face attendance system\Backend\known_faces_database\known_face_labels.npy", labels)
        print("'known_face_labels.npy' & 'known_face_encodings.npy' generated successfully .....")

    elif len(os.listdir(path)) == 0:
        print("Please Ensure that, you have some images to Process !!!!")


path = r"D:\Projects\face attendance system\Input_images"
generate(path)


'known_face_labels.npy' & 'known_face_encodings.npy' generated successfully .....


## Utils.py test  

In [3]:
# Backend\\util_functions\\utils.py

import datetime,calendar
import pandas as pd
import numpy as np
import os

def base_df(names_list,today,current_month_yr,csv_out_path):
    """ Takes Names list ; creates all columns for that month-csv """
    day, row, num_days = 1, [], calendar.monthrange(today.year, today.month)[1]
    
    for i in range(num_days):
        d = "0" + str(day) if len(str(day)) == 1 else str(day)  # if 1:'01'; 17:'17'
        row.append(d + "-" + current_month_yr)
        day += 1

    """ generates static df with name & day-wise column(default 0) """
    df = pd.DataFrame(0, columns=names_list, index=row)
    df.to_csv(csv_out_path, index=True)
    print("CSV created @ ", csv_out_path)
    #return df
##
# base_df(names_list,today,current_month_yr,csv_out_path)



def data_entry_verifier(df,name,names_list,threshold_in_time):
    """ Takes [name] , name , today & now; to apped data """
    name = name.title()
    row = datetime.datetime.today().day-1  # today's day as Row_id
    col = names_list.index(name)+1  # Name index as Col_id

    # ### Entry if person in-time & no duplicate entry allowed
    if datetime.datetime.now().time() <= threshold_in_time.time():  # if now <= threshold
        if df.iloc[row, col] == "0" or df.iloc[row, col] == 0:  # if in-time not-there then add (no duplicate data entry)
            now=datetime.datetime.now().strftime('%H:%M:%S')
            df.iloc[row, col] = now   # Append In-time in (row,col)
            print("Added {} @ {} !!!".format(name, now))
        else:
            print("{} Already added".format(name))
    elif datetime.datetime.now().time() > threshold_in_time.time():
        print("You are late !!!")
##
#data_entry_verifier(df,"Mark",names_list,threshold_in_time)



def csv_writer(name,threshold_in_time):
    
    ## parameters
    names_list = np.load(r"D:\Projects\face attendance system\Backend\known_faces_database\known_face_labels.npy").tolist()
    names_list = [i.title() for i in names_list]

    today = datetime.datetime.today()
    current_month_yr = today.strftime("%b-%Y")              # current month & year (eg- "May-2022" )
    threshold_in_time=threshold_in_time
    out_csv_name = current_month_yr + ".csv"
    csv_out_path = os.path.join(r"D:\Projects\face attendance system\Backend\database\master_csv", out_csv_name)

    
    
    if out_csv_name not in os.listdir(r"D:\Projects\face attendance system\Backend\database\master_csv"):
        base_df(names_list,today,current_month_yr,csv_out_path)    # create base structure for csv
        df = pd.read_csv(csv_out_path)
        data_entry_verifier(df,name,names_list,threshold_in_time)
        df.to_csv(csv_out_path, index=False)
    else:
        df = pd.read_csv(csv_out_path)
        data_entry_verifier(df,name,names_list,threshold_in_time)
        df.to_csv(csv_out_path, index=False)

##
threshold_in_time=datetime.datetime.strptime("23:30:00",'%H:%M:%S')          # threshold In-time
csv_writer('Jeff Bezos' , threshold_in_time)

Jeff Bezos Already added


In [35]:
## Test on all names
import time
names_list = np.load(r"D:\Projects\face attendance system\known_faces_database\known_face_labels.npy").tolist()
names_list = [i.title() for i in names_list]
    
for i in names_list:
    csv_writer(i, threshold_in_time)
    time.sleep(.5)

Added Abdul Kalam @ 12:54:19 !!!
Added Barack Obama @ 12:54:20 !!!
Added Elon Musk @ 12:54:20 !!!
Added Guido Van Rossum @ 12:54:21 !!!
Jeff Bezos Already added
Added Johnny Depp @ 12:54:22 !!!
Added Larry Page @ 12:54:22 !!!
Added Leonardo Dicaprio @ 12:54:23 !!!
Added Mark @ 12:54:23 !!!
Added Satya Nadella @ 12:54:24 !!!
Added Sundar Pichai @ 12:54:24 !!!
Added Warren Buffett @ 12:54:25 !!!


## From Images

In [107]:
# Backend\\main_from_img_dir.py
"""Note:
    - Uses image path to Append in Attendance-Sheet 
"""

import numpy as np
import cv2,datetime
import face_recognition as fr

def get_attendance_from_image(path,threshold_in_time):
    """ Takes Image path => Preprocess => extract-face-encoding => compare with known face encodings => retun match & Append to csv """
    
    # Load Known face-encodings & labels
    known_encodings = np.load(r"D:\Projects\face attendance system\Backend\known_faces_database\known_face_encodings.npy")  # load stored face-encodings
    known_labels = np.load(r"D:\Projects\face attendance system\Backend\known_faces_database\known_face_labels.npy")        # load stored labels

    # Preprocess test-image
    img=cv2.imread(path)
    resized_img = cv2.resize(img, (0, 0), None, 0.5, 0.5)       # resized images
    resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)  # BGR to RGB

    # Extract face-encoding from test-image
    face_loc = fr.face_locations(resized_img)                    # detect all faces-location in frame
    test_face_encode = fr.face_encodings(resized_img, face_loc)  # face encodings on frame


    # Compare test-face-encoding with known-face-encoding
    for encode_face,face_location in zip(test_face_encode,face_loc):
        matches = fr.compare_faces(known_encodings,encode_face)        # compares new-face-encodeing with known-faces-encodings(returns Boolean value for each [label])
        face_distance = fr.face_distance(known_encodings,encode_face)  # compares frame face-distance to known-faces-distance
        match_id = np.argmin(face_distance)                            # index of best matching label_index

        if matches[match_id]:  # If known-face matches to face in frame
            name = known_labels[match_id].title()
            print("Matched With '{}'".format(name))
            csv_writer(name , threshold_in_time)
        else:
            print("No Match Found")
   

##
test_img_path=r"D:\Projects\face attendance system\test data\download (2).jpg"
threshold_in_time=datetime.datetime.strptime("23:30:00",'%H:%M:%S')          # threshold In-time

get_attendance_from_image(test_img_path,threshold_in_time)

Matched With 'Leonardo Dicaprio'
Leonardo Dicaprio Already added


## From Webcam


In [4]:
## Backend\\main_from_webcam.py

import numpy as np
import cv2, datetime
import face_recognition as fr

# Load Known face-encodings & labels
known_encodings = np.load(r"D:\Projects\face attendance system\Backend\known_faces_database\known_face_encodings.npy")  # load stored face-encodings
known_labels = np.load(r"D:\Projects\face attendance system\Backend\known_faces_database\known_face_labels.npy")        # load stored labels
threshold_in_time = datetime.datetime.strptime("23:30:00", '%H:%M:%S')  # threshold In-time

## live
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    resized_img = cv2.resize(img, (0, 0), None, 0.5, 0.5)  # resized images
    resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)  # BGR to RGB

    # Extract face-encoding from test-image
    face_loc = fr.face_locations(resized_img)  # detect all faces-location in frame
    test_face_encode = fr.face_encodings(resized_img, face_loc)  # face encodings on frame

    # Compare test-face-encoding with known-face-encoding
    for encode_face, face_location in zip(test_face_encode, face_loc):
        matches = fr.compare_faces(known_encodings,encode_face)  # compares new-face-encodeing with known-faces-encodings(returns Boolean value for each [label])
        face_distance = fr.face_distance(known_encodings,encode_face)  # compares frame face-distance to known-faces-distance
        match_id = np.argmin(face_distance)  # index of best matching label_index

        if matches[match_id]:  # If known-face matches to face in frame
            name = known_labels[match_id].title()
            print("Matched With '{}'".format(name))
            csv_writer(name, threshold_in_time)
        else:
            print("No Match Found")

    cv2.imshow('Output', img)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
#cv2.destroyAllWindows()


No Match Found
